In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

IMG_SIZE = 50

x = tf.placeholder(tf.float32, shape = [None, IMG_SIZE*IMG_SIZE])
y_ = tf.placeholder(tf.float32, shape = [None, 2])

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

In [3]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides = [1,1,1,1], padding = 'SAME')

def max_pool2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [4]:
def get_product_shape(x):
    b = 1
    list_x = x.get_shape().as_list()
    for i in list_x[1:]:
        b = b * i
        
    return b

In [5]:
def convolution_layer(x,shape):
    W_conv1 = weight_variable(shape)
    b_conv1 = bias_variable([shape[-1]])
    h_conv1 = tf.nn.relu(conv2d(x,W_conv1) + b_conv1)
    h_pool1 = max_pool2x2(h_conv1)
    
    return h_pool1

def fully_connected_layer(pool_layer,shape,flatten = False):
    W_fc1 = weight_variable([get_product_shape(pool_layer), shape])
    b_fc1 = bias_variable([shape])
    if flatten == True:
        pool_layer = tf.reshape(pool_layer,[-1,get_product_shape(pool_layer)])
        
    h_fc1 = tf.nn.relu(tf.matmul(pool_layer, W_fc1) + b_fc1)
    
    return h_fc1

In [6]:
x_image = tf.reshape(x,[-1,IMG_SIZE,IMG_SIZE,1])
conv_layer_1 = convolution_layer(x_image,[5,5,1,32])
conv_layer_2 = convolution_layer(conv_layer_1,[5,5,32,64])

conv_layer_3 = convolution_layer(conv_layer_2,[5,5,64,32])
conv_layer_4 = convolution_layer(conv_layer_3,[5,5,32,64])

conv_layer_5 = convolution_layer(conv_layer_4,[5,5,64,32])
conv_layer_6 = convolution_layer(conv_layer_5,[5,5,32,64])


fc1 = fully_connected_layer(conv_layer_6,1024,flatten = True)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(fc1,keep_prob)

y_conv = fully_connected_layer(h_fc1_drop,2)

In [7]:
import os
import cv2
from tqdm import tqdm
from random import shuffle
import numpy as np

TRAIN_DIR = 'C:/Users/kaustabh/Downloads/train/train'
TEST_DIR = 'C:/Users/kaustabh/Downloads/test/test'

In [8]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [9]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
   
    return training_data

In [10]:
train_data = np.load('train_data.npy')

In [11]:
X_train = np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE*IMG_SIZE)
Y_train =np.asarray([i[1] for i in train_data])

In [12]:
EPOCH = 10000
BATCH_SIZE = 100
TRAIN_DATASIZE,_,_,_ = X_train.shape
PERIOD = TRAIN_DATASIZE/BATCH_SIZE

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
j = 0

sess.run(tf.global_variables_initializer())
for i in range(EPOCH):
        
    batch_X, batch_Y = next_batch(100,X_train,Y_train)      
    if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict = {x: batch_X, y_: batch_Y, keep_prob: 1.0})
            print('Step', i , 'Training accuracy', train_accuracy)
    
    train_step.run(feed_dict = {x: batch_X, y_: batch_Y, keep_prob: 0.5})     

Step 0 Training accuracy 0.47


In [12]:
def next_batch(num, data, labels):
    
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)